## Test Mel Spectogram

### Import library

In [2]:
import librosa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import cv2
import os

### Spectogram

In [3]:
def mp3_to_melspec(mp3_file, output_path, *, sr=22050, duration=30, n_fft=2048, n_mels=244, hop_length=2713):

    y, sr = librosa.load(mp3_file, sr=sr, duration=duration)

    # calculete mel-spectogram
    mel_spec = librosa.feature.melspectrogram(
    y=y, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels
    )
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    normalized = (mel_spec_db - np.min(mel_spec_db)) / (np.max(mel_spec_db)-np.min(mel_spec_db))  #

    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    librosa.display.specshow(normalized, sr=sr, hop_length=hop_length)
    plt.savefig(str(output_path), bbox_inches='tight', pad_inches=0)
    plt.close()

In [4]:
AUDIO_PATH = Path.cwd() / 'project_data' / 'audio_data'
SPECT_PATH = Path.cwd() / 'project_data' / 'spectrogram_data'
mp3_files = AUDIO_PATH.glob('*.mp3')

for file in mp3_files:
    file_stem = file.stem
    output_file = SPECT_PATH / f"{file_stem}.png"
    mp3_to_melspec(file, output_file)

In [5]:
df = pd.read_csv('project_data/tracks.csv')

df['filename'] = df['track_id'].apply(lambda x: f'{x:06d}.png')
new_data = df[['filename', 'genre']]
new_data.to_csv('dataset/spectrogram.csv', index=False)